## Test `NetworkModel`

In [1]:
import numpy as np
from coremdlr.models import NetworkModel
from coremdlr.datasets import DepthSequenceGenerator, FaciesDataset

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
fdset = FaciesDataset(["205-21b-3", "204-20-6a", "204-20-1Z", "204-24a-6"],
                    test_wells=["204-19-6"],
                    features=["image"],
                    label_resolution=32)

fdset.load_or_generate_data()

Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('image', (3881, 32, 600, 3))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('image', (3542, 32, 600, 3))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1929,)), ('top', (1929,)), ('base', (1929,)), ('image', (1929, 32, 600, 3))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('image', (13034, 32, 600, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1981,)), ('top', (1981,)), ('base', (1981,)), ('image', (1981, 32, 600, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [3]:
print(np.bincount(fdset.y_train) / fdset.y_train.size)
print(np.bincount(fdset.y_test) / fdset.y_test.size)

[0.00352899 0.27673546 0.15934066 0.19686411 0.02045922 0.34307156]
[0.04896517 0.47147905 0.15497224 0.14386673 0.         0.18071681]


In [4]:
bcnn_model_args = {
    'feature' : 'image',
    'network' : 'bilinear_cnn',
    'sequence_size' : 12,
    'network_args' : {
        'conv1x1' : 32,
        'backbone_cnn' : 'vgg16',
        'lstm_features' : None,
        'dropout_rate' : 0.25,
        'apply_rowwise' : True
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 10.
    },
    'loss' : {'ordinal_squared_error': 0.5,
              'categorical_crossentropy': 1.0}
}

bcnn_model = NetworkModel(fdset, model_args=bcnn_model_args)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 384, 600, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 12, 18, 512)  14714688    input_2[0][0]                    
__________________________________________________________________________________________________
reduce_A (Conv2D)               (None, 12, 18, 32)   16416       vgg16[1][0]                      
__________________________________________________________________________________________________
reduce_B (Conv2D)               (None, 12, 18, 32)   16416       vgg16[1][0

In [5]:
fit_args = {
    'batch_size' : 4,
    'step_size' : bcnn_model.sequence_size - 2,
    'epochs' : 3,
}

bcnn_model.fit(fdset, **fit_args)

Shapes of `(batch_X, batch_y)`: (4, 384, 600, 3), (4, 12, 6)
Epoch 1/3
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
559/559 [==============================] - 94s 168ms/step - loss: 2.4590 - acc: 0.5190 - val_loss: 2.6782 - val_acc: 0.4721
Epoch 2/3
559/559 [==============================] - 87s 156ms/step - loss: 2.8212 - acc: 0.3283 - val_loss: 3.2088 - val_acc: 0.1815
Epoch 3/3
559/559 [==============================] - 87s 156ms/step - loss: 2.4421 - acc: 0.4549 - val_loss: 2.2893 - val_acc: 0.5859


2.289321176594284

In [6]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(bcnn_model.network).create(prog='dot', format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [ ]:
strip = fdset.wells[0]._make_striplog()

In [ ]:
import pandas as pd
from io import StringIO

pd.read_csv(StringIO(strip.to_csv()))

In [ ]:
from striplog import Decor, Component, Legend

###++++++++++++++++++++###
### Facies Definitions ###
###++++++++++++++++++++###

facies = {
    'nc': Component({'lithology': 'none', 'train':'n'}),
    'bs': Component({'lithology': 'sandstone', 'train': 'n'}),
    's': Component({'lithology': 'sandstone', 'train':'y'}),
    'is': Component({'lithology': 'clay-prone sandstone', 'train':'y'}),
    'ih': Component({'lithology': 'sandy mudstone', 'train':'y'}),
    'sh': Component({'lithology': 'mudstone', 'train':'y'}),
    't': Component({'lithology': 'turbidite', 'train':'y'}),
}

lithologies = [c.lithology for c in facies.values()]

def lithology_to_key(lithology_str):
    try:
        litho = lithology_str.split(',')[0].lower()
        idx = lithologies.index(litho)
        return list(facies.keys())[idx]
    except IndexError:
        raise ValueError('{} is not present in lithologies: {}'.format(lithology_str, lithologies))

In [ ]:
df = pd.read_csv(StringIO(strip.to_csv()))
df.Component = df.Component.apply(lithology_to_key)
df

In [ ]:
df.columns = ['top', 'base', 'lithology']
df.to_csv()

In [ ]:
from coremdlr.datasets import strip_config

In [ ]:
list(strip_config.values())[0].lithology

In [ ]:
fdset.wells[0]._y.size

In [ ]:
fdset.wells[0]._image.max()

In [ ]:
fdset.wells[0].data_path

In [ ]:
img = np.load('/home/administrator/Dropbox/core_data/facies/train_data/204-19-6_image.npy')
img.min(), img.max()

In [ ]:
a = np.random.rand(10,5)
a

In [ ]:
a[np.where(a > 0.5)]

In [ ]:
fdset.X_train['pseudoGR'].shape